In [1]:
import os, sys, glob
import itertools
import math

import gsd.hoomd
import hoomd
from hoomd import *
from hoomd import sph
from hoomd.sph import _sph
import numpy as np
import matplotlib
import numpy

# import itertools
from datetime import datetime
import export_gsd2vtu 

%matplotlib inline
matplotlib.style.use('ggplot')
import matplotlib_inline

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

ImportError: generic_type: type "IntegrationMethodListSPHIntegratorTwoStep_WC2_T" is already registered!

In [ ]:
fn = os.path.join(os.getcwd(), 'aggregates.gsd')
![ -e "$fn" ] && rm "$fn"

fn = os.path.join(os.getcwd(), 'lattice.gsd')
![ -e "$fn" ] && rm "$fn"

In [ ]:
# This is not intended as a full tutorial on fresnel - see the
# fresnel user documentation (https://fresnel.readthedocs.io/) if you would like to learn more.

import warnings

import fresnel
import IPython
import packaging.version

device = fresnel.Device()
tracer = fresnel.tracer.Path(device=device, w=500, h=500)

FRESNEL_MIN_VERSION = packaging.version.parse("0.13.0")
FRESNEL_MAX_VERSION = packaging.version.parse("0.14.0")


def render(snapshot):
    if ('version' not in dir(fresnel) or packaging.version.parse(
            fresnel.version.version) < FRESNEL_MIN_VERSION
            or packaging.version.parse(
                fresnel.version.version) >= FRESNEL_MAX_VERSION):
        warnings.warn(
            f"Unsupported fresnel version {fresnel.version.version} - expect errors."
        )
    central_color = fresnel.color.linear([252 / 255, 41 / 255, 0 / 255])
    constituent_color = fresnel.color.linear([93 / 255, 210 / 255, 252 / 255])
    green = fresnel.color.linear([0 / 255, 255 / 255, 0 / 255])
    violet = fresnel.color.linear([238 / 255, 130 / 255, 238 / 255])

    
    L = snapshot.configuration.box[0]
    scene = fresnel.Scene(device)
    geometry = fresnel.geometry.Sphere(scene,
                                       N=len(snapshot.particles.position)
                                      )
    geometry.material = fresnel.material.Material(color=[0, 0, 0],
                                                  roughness=0.5,
                                                  primitive_color_mix=1.0)
    geometry.position[:] = snapshot.particles.position[:]
    geometry.radius[snapshot.particles.typeid[:] == 0] = 0.0061125
    geometry.color[snapshot.particles.typeid[:] == 0] = central_color
    geometry.radius[snapshot.particles.typeid[:] == 1] = 0.0125
    geometry.color[snapshot.particles.typeid[:] == 1] = constituent_color
    geometry.radius[snapshot.particles.typeid[:] == 2] = 0.025
    geometry.color[snapshot.particles.typeid[:] == 2] = green
    geometry.radius[snapshot.particles.typeid[:] == 3] = 0.027
    geometry.color[snapshot.particles.typeid[:] == 3] = violet
    
    geometry.outline_width = 0.002
    box = fresnel.geometry.Box(scene, [L, L, L, 0, 0, 0], box_radius=.002)

    scene.lights = [
        fresnel.light.Light(direction=(0, 0, 1),
                            color=(0.8, 0.8, 0.8),
                            theta=math.pi),
        fresnel.light.Light(direction=(1, 1, 1),
                            color=(1.1, 1.1, 1.1),
                            theta=math.pi / 3)
    ]
    scene.camera = fresnel.camera.Orthographic(position=(L * 2, L, L * 2),
                                               look_at=(0, 0, 0),
                                               up=(0, 1, 0),
                                               height=L * 1.4 + 1)
    scene.background_alpha = 1
    scene.background_color = (1, 1, 1)
    return IPython.display.Image(tracer.sample(scene, samples=500)._repr_png_())

In [ ]:
# fluid and particle properties
num_length          = 10
lref                = 1.0               # [m]
voxelsize           = lref/num_length
dx                  = voxelsize
specific_volume     = dx * dx * dx
rho0                = 1000.0
mass                = rho0 * specific_volume
# refvel              = 10
fx                  = 0.1                # [m/s]
viscosity           = 0.01               # [Pa s]
print(dx)

In [ ]:
# aggregate properties

#sphere1
rs1  = lref*0.1
xs1  = 0.25*lref
ys1  = 0.0*lref
zs1  = 0.0*lref

#sphere2
rs2  = lref*0.1
xs2  = -0.25*lref
ys2  = 0.0*lref
zs2  = 0.0*lref

# aggregate array
ra = np.array([rs1, rs2])
xs = np.array([xs1, xs2])
ys = np.array([ys1, ys2])
zs = np.array([zs1, zs2])

rhos0 = 10.0
masss = rhos0 * specific_volume

In [ ]:
# get kernel properties
kernel  = 'WendlandC4'
slength = hoomd.sph.kernel.OptimalH[kernel]*dx       # m
rcut    = hoomd.sph.kernel.Kappa[kernel]*slength     # m

In [ ]:
# particles per Kernel Radius
part_rcut  = math.ceil(rcut/dx) 
#part_depth = math.ceil(2.5 * hoomd.sph.kernel.Kappa[kernel] * rcut/dx) 

In [ ]:
# get simulation box sizes etc.
nx, ny, nz = int(num_length), int(num_length + (3*part_rcut)), int(num_length)
lx, ly, lz = float(nx) * voxelsize, float(ny) * voxelsize, float(nz) * voxelsize
# box dimensions
box_lx, box_ly, box_lz = lx, ly, lz
#print('box_lx:', box_lx)
#print('box_ly:', box_ly)
#print('box_lz:', box_lz)

In [ ]:
# Number of Particles in regular grid
n_particles = nx * ny * nz 
print('number_particles:', n_particles)

In [ ]:
# define meshgrid and add properties
x, y, z = np.meshgrid(*(np.linspace(-box_lx / 2 + dx/2, box_lx / 2 - dx/2, nx, endpoint=True),),
                      *(np.linspace(-box_ly / 2 + dx/2, box_ly / 2 - dx/2, ny, endpoint=True),),
                      *(np.linspace(-box_lz / 2 + dx/2, box_lz / 2 - dx/2, nz, endpoint=True),))

positions = np.array((x.ravel(), y.ravel(), z.ravel())).T
print(positions)
print(len(positions))

In [ ]:
n_aggregates = len(xs)
print(n_aggregates)
n_particles_aggregate = np.zeros((n_aggregates), dtype = np.int32)
type_aggregate = np.zeros((len(positions)), dtype = np.int32)
delete_global_pos = np.zeros((len(positions)), dtype = np.int32)

# type aggregate eventually necessary
for i in range(len(positions)):
    xi,yi,zi  = positions[i][0], positions[i][1], positions[i][2]
    for k in range(n_aggregates):
        if ( (xi-xs[k])**2 + (yi-ys[k])**2 + (zi-zs[k])**2 < ra[k]**2 ):
            #tid[i]    = 3
            #dens[i] = rhos0
            n_particles_aggregate[k] = n_particles_aggregate[k] + 1
            type_aggregate[i] = k + 1
            delete_global_pos[i] = 1

            
delete_global_pos = np.where(delete_global_pos == 1)
consitutent_particle_count = len(delete_global_pos)
print(delete_global_pos)


# save the deleted positions in another array
#constitutent_particles_pos = np.zeros((consitutent_particle_count, 4),dtype = np.float32)
#for i in range(n_aggregates):
#    temp_aggregate_array = np.where(delete_global_pos == i+1)
#    constitutent_particles_pos[]
    
initial_positions = positions 
print(initial_positions)
print(len(initial_positions))

#delete positions where solid aggregates are
#delete_global_pos = np.where(delete_global_pos == 1)
positions = np.delete(positions, delete_global_pos, 0)
n_particles = len(positions)
print(n_particles)

In [ ]:
velocities = np.zeros((positions.shape[0], positions.shape[1]), dtype = np.float32)
masses     = np.ones((positions.shape[0]), dtype = np.float32) * mass
slengths   = np.ones((positions.shape[0]), dtype = np.float32) * slength
densities  = np.ones((positions.shape[0]), dtype = np.float32) * rho0

In [ ]:
snapshot = gsd.hoomd.Frame()
snapshot.configuration.box     = [box_lx, box_ly, box_lz] + [0, 0, 0]
snapshot.particles.N           = n_particles
snapshot.particles.position    = positions
snapshot.particles.typeid      = [0] * n_particles
snapshot.particles.types       = ['F','S','Center','A']
snapshot.particles.velocity    = velocities
snapshot.particles.mass        = masses
snapshot.particles.slength     = slengths
snapshot.particles.density     = densities

In [ ]:
x    = snapshot.particles.position[:]
tid  = snapshot.particles.typeid[:]
vels = snapshot.particles.velocity[:]
dens = snapshot.particles.density[:]
for i in range(len(x)):
    xi,yi,zi  = x[i][0], x[i][1], x[i][2]
    tid[i]    = 0
    # solid walls 
    if ( yi < -0.5 * lref or yi > 0.5 * lref):
        tid[i] = 1
print(x)

In [ ]:
snapshot.particles.typeid[:]     = tid
snapshot.particles.velocity[:]   = vels

In [ ]:
#n_aggregates = len(xs)
#print(n_aggregates)
#aggregates = np.zeros((len(xi)))
#n_particles_aggregate = np.zeros((n_aggregates), dtype = np.int32)
#type_aggregate = np.zeros((len(x)), dtype = np.int32)

#for i in range(len(x)):
#    xi,yi,zi  = x[i][0], x[i][1], x[i][2]
#    for k in range(n_aggregates):
#        if ( (xi-xs[k])**2 + (yi-ys[k])**2 + (zi-zs[k])**2 < ra[k]**2 ):
#            tid[i]    = 3
#            dens[i] = rhos0
#           n_particles_aggregate[k] = n_particles_aggregate[k] + 1
#           type_aggregate[i] = k + 1

In [ ]:
print(n_particles_aggregate)
print(type_aggregate)
print(n_aggregates)

In [ ]:
# dict for solid aggregate positions
aggregates = {}
# aggregate mass
mass_aggregate = np.zeros(n_aggregates)
# aggregate center of mass 
com_aggregate = np.zeros((n_aggregates,3))

# writing agggregate positions to dict - use initial positions
x_init = initial_positions
for i in range(len(n_particles_aggregate)):   
    l = 0
    aggregate_temp = np.zeros((n_particles_aggregate[i],3))
    for k in range(len(type_aggregate)):
        if (type_aggregate[k] == i + 1):    
            aggregate_temp[l][:] = x_init[k][0], x_init[k][1], x_init[k][2]
            l = l + 1
    aggregates['Aggregate' + str(i+1)] = aggregate_temp

# calculate aggregate mass    
for i in range(n_aggregates):
    mass_aggregate[i] = n_particles_aggregate[i] * masss
    
print(mass_aggregate)
    
# calculate center of mass                
for i in range(n_aggregates):
    aggregate_temp = aggregates['Aggregate' + str(i+1)]
    print(aggregate_temp)
    for k in range(n_particles_aggregate[i]):
        com_aggregate[i][0] = com_aggregate[i][0] + masss * aggregate_temp[k][0]
        com_aggregate[i][1] = com_aggregate[i][1] + masss * aggregate_temp[k][1]
        com_aggregate[i][2] = com_aggregate[i][2] + masss * aggregate_temp[k][2]
    com_aggregate[i][:] = com_aggregate[i][:] / mass_aggregate[i]
    # add numerical zero

# calculate position concerning center of mass
for i in range(len(n_particles_aggregate)):   
    print(i)
    l = 0
    aggregate_temp = np.zeros((n_particles_aggregate[i],3))
    for k in range(len(type_aggregate)):
        if (type_aggregate[k] == i + 1):    
            aggregate_temp[l][:] = com_aggregate[i][0] - x_init[k][0], com_aggregate[i][1] - x_init[k][1], com_aggregate[i][2] - x_init[k][2]
            l = l + 1
    aggregates['Aggregate' + str(i+1)] = aggregate_temp


print(com_aggregate)


# calculate moment of inertia   
#for i in range(n_aggregates):
    #
    
# aggregate velocity
vels_aggregate = np.zeros((com_aggregate.shape[0], com_aggregate.shape[1]), dtype = np.float32)

# aggregate slengths
slengths_aggregate   = np.ones((com_aggregate.shape[0]), dtype = np.float32) * slength

# aggregate densities
dens_aggregate  = np.ones((com_aggregate.shape[0]), dtype = np.float32) * rhos0

# aggregate typeid
tid_aggregate = np.ones((com_aggregate.shape[0]), dtype = np.float32) * 2

In [ ]:
# add aggregates to snapshot
n_particles =  n_particles + n_aggregates
positions = np.concatenate((positions, com_aggregate))
tid  = np.concatenate((tid, tid_aggregate))
vels = np.concatenate((vels, vels_aggregate))
dens = np.concatenate((dens, dens_aggregate))
slengths = np.concatenate((slengths, slengths_aggregate))
masses = np.concatenate((masses, mass_aggregate))

In [ ]:
snapshot.particles.N           = n_particles
snapshot.particles.position    = positions
snapshot.particles.typeid      = tid
snapshot.particles.velocity    = vels
snapshot.particles.mass        = masses
snapshot.particles.slength     = slengths
snapshot.particles.density     = dens

In [ ]:
# Define groups/filters
filterfluid  = hoomd.filter.Type(['F']) # is zero
filtersolid  = hoomd.filter.Type(['S']) # is one
filteraggregate  = hoomd.filter.Type(['A']) # is two
filtercenter  = hoomd.filter.Type(['Center']) # is three
filterall    = hoomd.filter.All()

In [ ]:
kernel_obj = hoomd.sph.kernel.Kernels[kernel]()
kappa      = kernel_obj.Kappa()
# Neighbor list
nlist = hoomd.nsearch.nlist.Cell(buffer = rcut*0.05, rebuild_check_delay = 1, kappa = kappa)

# Equation of State
eos = hoomd.sph.eos.Tait()
eos.set_params(rho0,0.01)

In [ ]:
with gsd.hoomd.open(name='aggregates.gsd', mode='xb') as f:
    f.append(snapshot)

In [ ]:
model = hoomd.sph.sphmodel.SuspensionFlow(kernel = kernel_obj,
                                           eos    = eos,
                                           nlist  = nlist,
                                           fluidgroup_filter = filterfluid,
                                           solidgroup_filter = filtersolid,
                                           aggregategroup_filter = filteraggregate,
                                           comgroup_filter = filtercenter)

In [ ]:
model.mu = viscosity
model.rhoS0 = rhos0
model.gx = 1.0

In [ ]:
# create list
aggregate_types = {}
for i in range(n_aggregates):
    print(i)
    temp_list = []
    for j in range(n_particles_aggregate[i]):
        temp_list.extend('A')
    aggregate_types['Aggregate' + str(i+1)] = temp_list
    
#print(aggregate_types)
#print(aggregate_types['Center'])

In [ ]:
# create solid aggregates
#for i in range(n_aggregates):
#    model.body['Aggregate' + str(i+1)] = {
#    "constituent_types": aggregate_types['Aggregate' + str(i+1)],
#    "positions": aggregates['Aggregate' + str(i+1)],
#    }


model.body['Center'] = {
"constituent_types": aggregate_types['Aggregate1'],
"positions": aggregates['Aggregate1'],
    }
print(model.body['Center'])
#print(model.body['Aggregate2'])
#print(model.body['Center'])

In [ ]:
sim = hoomd.Simulation(device=hoomd.device.CPU(), seed=4)
sim.create_state_from_gsd(filename='aggregates.gsd')
#render(sim.state.get_snapshot())

In [ ]:
print(model)
model.create_bodies(sim.state)

In [ ]:
#render(sim.state.get_snapshot())

In [ ]:
with sim.state.cpu_local_snapshot as snapshot:
    typeid = snapshot.particles.typeid
    print(len(typeid))
    test = np.where(typeid == 3)
    print(test)
    print(len(test))
    pos = snapshot.particles.position
    for i in range(len(test)):
        print(pos[test[i]])
    #pos[2194][:] = ( 0.25,  0.25,    0.)
    #snapshot.particles.position = pos

In [ ]:
#render(sim.state.get_snapshot())

In [ ]:
print(model.body['Center'])
#sim.create_state_from_gsd(filename='aggregates.gsd')


In [ ]:
#dt = model.compute_dt(lref, refvel, dx, drho)
integrator = hoomd.sph.Integrator(dt=0.00005, kernel = kernel, eos = eos)
sim.operations.integrator = integrator

In [ ]:
integrator.rigid = model

In [ ]:
#print(integrator.rigid)

In [ ]:
velocityverlet = hoomd.sph.methods.VelocityVerletBasic(filter=filterfluid, densitymethod = 'SUMMATION')

In [ ]:
solidaggregateintegrator = hoomd.sph.methods.SolidAggregateIntegrator(filter=filtercenter, densitymethod = 'SUMMATION')

In [ ]:
integrator.methods.append(velocityverlet)
integrator.methods.append(solidaggregateintegrator)
integrator.forces.append(model)

In [ ]:
#print(integrator.methods.getIntegrationMethods)